# DEEPX Tutorial 04 - DX STREAM workflow

This forth tutorial intruduces DX-STREAM overview and end-to-end workflow for a custom AI model with DX-STREAM.

 - Understand what DX-STREAM is and where it fits in the DEEPX SDK
 - Learn the core elements (DxPreprocess → DxInfer → DxPostprocess → DxTracker → DxOsd)
 - Implement custom preprocess/postprocess libraries to support customized AI models

>This tutorial is based on dx-all-suite v2.0.0, released in September 2025.

## DX-STREAM Overview
A set of GStreamer elements for Vision AI on DEEPX NPUs.
```text
Source → (decodebin) → DxPreprocess → DxInfer → DxPostprocess → DxTracker → DxOsd → Sink
```
- **DxPreprocess**: resize/ROI/color space, optional custom library
- **DxInfer**: (Inference) runs .dxnn model, links to preprocess via `preprocess-id`
- **DxPostprocess**: decode tensors → metadata, optional custom library
- **DxTracker**: assign stable IDs (e.g., OC_SORT)
- **DxOsd**: (On-Screen Display) draw boxes/labels/poses/segmentation onto frames

  ![img](assets/dx-stream-pipeline.png)

For more details, download DX_STREAM User Guide from 👉 [here](https://developer.deepx.ai/?files=MjUxNw==)!

## Prerequisites:

1. Move to `dx_stream` directory:

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-runtime/dx_stream"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-runtime/dx_stream

2. Download required models and sample videos by running the following command:

In [ ]:
# Assets (models + videos) are downloaded and placed in the assets/ directory.
!./setup.sh

3. Verify required plugins - following dx-plugins should be displayed:
  - dxgather: DxGather
  - dxinfer: DXInfer
  - dxinputselector: DXInputSelector
  - dxmsgbroker: DXMsgBroker
  - dxmsgconv: DXMsgConv
  - dxosd: DXOsd
  - dxoutputselector: DXOutputSelector
  - dxpostprocess: DXPostprocess
  - dxpreprocess: DXPreprocess
  - dxrate: DXRate
  - dxtracker: DXTracker

In [ ]:
!gst-inspect-1.0 dxstream || echo 'dxstream plugin not found'

## Quick Starts with run_demo scripts in DX_STREAM

1. Let's run the YOLOv7 pipeline with video input:
   - `Note`: You can stop the pipeline by clicking the stop button ('■') above!

In [ ]:
# video file as an input
VIDEO_SRC='dx_stream/samples/videos/doughnut.mp4'

# GStreamer pipeline configuration
!gst-launch-1.0 filesrc location=$VIDEO_SRC ! decodebin ! \
                   dxpreprocess config-file-path=dx_stream/configs/Object_Detection/YoloV7/preprocess_config.json ! queue ! \
                   dxinfer config-file-path=dx_stream/configs/Object_Detection/YoloV7/inference_config.json ! queue ! \
                   dxpostprocess config-file-path=dx_stream/configs/Object_Detection/YoloV7/postprocess_config.json ! queue ! \
                   dxosd width=1280 height=720 ! queue ! \
                   videoconvert ! fpsdisplaysink sync=false

2. Let's learn about the definition and usage of each plugin using **gst-inspect-1.0**.

In [ ]:
# Check dxpreprocess plugin
!gst-inspect-1.0 dxpreprocess
!echo "==============[ Configuration Start ]=============="
!cat dx_stream/configs/Object_Detection/YoloV7/preprocess_config.json
!echo "==============[  Configuration End  ]=============="

In [ ]:
# Check dxinfer plugin
!gst-inspect-1.0 dxinfer
!echo "==============[ Configuration Start ]=============="
!cat dx_stream/configs/Object_Detection/YoloV7/inference_config.json
!echo "==============[  Configuration End  ]=============="

In [ ]:
# Check dxpostprocess plugin
!gst-inspect-1.0 dxpostprocess
!echo "==============[ Configuration Start ]=============="
!cat dx_stream/configs/Object_Detection/YoloV7/postprocess_config.json
!echo "==============[  Configuration End  ]=============="

In [ ]:
# Check dxosd plugin
!gst-inspect-1.0 dxosd

## Run Demo Script

There is a **`run_demo.sh`** script file in the dx_stream path, which contains main examples of using dx_stream.

This script calls the following script based on the user's selection (from 0 to 7):

In [ ]:
# Check the demo list - Total 8 GStreamer demos
!tail -n 15 run_demo.sh

Let's run each demo by changing the argument from '0' to '7':
   - `Note`: You can stop the pipeline by clicking the stop button ('■') above!

![img](assets/pipline-single-detection.png)

In [ ]:
!./run_demo.sh <<< 0 # single_network/object_detectionS

In [ ]:
!./run_demo.sh <<< 1 # single_network/face_detection

In [ ]:
!./run_demo.sh <<< 3 # single_network/pose_estimation

In [ ]:
!./run_demo.sh <<< 4 # single_network/semantic_segmentation

![img](assets/pipline-single-tracking.png)

In [ ]:
!./run_demo.sh <<< 2 # tracking/run_YOLOV5S_tracker

![img](assets/pipeline-multi-stream.png)

In [ ]:
!./run_demo.sh <<< 5 # multi_stream/run_multi_stream_YOLOV5S

In [ ]:
!./run_demo.sh <<< 6 # rtsp/run_RTSP

![img](assets/pipeline-secondary.png)

In [ ]:
!./run_demo.sh <<< 7 # secondary_mode/run_secondary_mode

## Writeing Your Own Application

This chapter describes how to integrate a custom AI model and implement user-defined logic within the
DX-STREAM pipeline.

This guide focuses on how to configure and integrate custom logic into the DX-STREAM pipeline using
modular elements such as DxPreprocess, DxInfer, and DxPostprocess.

We will reuse the custom model (Forklift & Worker detector) in the previous tutorial-03. Therefore, need to customize the dxpostporcess plugin.

![img](assets/custom-pipeline.png)

### 1. Download complied DXNN file from the following link:

In [ ]:
!wget "cs.deepx.ai/_deepx_fae_archive/dx-tutorials/yolov7-forklift-person.dxnn"

### 2. Modify "dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp" file:

In [ ]:
DIFF_TEXT = r"""diff --git a/dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp b/dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp
index 7d048e9..26125ca 100644
--- a/dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp
+++ b/dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp
@@ -58,21 +58,11 @@ struct YoloConfig {
     float nms_threshold = 0.4f;      // IoU threshold for NMS

     // Number of classes in your dataset
-    int num_classes = 80;
+    int num_classes = 2;

     // COCO dataset class names (modify for your dataset)
     std::vector<std::string> class_names = {
-        "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
-        "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
-        "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
-        "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
-        "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
-        "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
-        "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
-        "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
-        "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse",
-        "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
-        "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
+        "Forklift", "Worker"
     };
 };

"""

# 파일로 저장
with open("dx_stream_update.diff", "w", encoding="utf-8") as f:
    f.write(DIFF_TEXT)

print("Saved: dx_stream_update.diff (bytes)", len(DIFF_TEXT))

### 3. Apply the modified code by using git apply command:

In [ ]:
!git apply --whitespace=fix dx_stream_update.diff

### 4. Re-build DX_STREAM libraries:

In [ ]:
!source ../venv-dx-runtime/bin/activate && ./build.sh

### 5. Create configuration json to load your custom model (yolov7-forklift-person.dxnn)

In [ ]:
import json, os
yolov7_custom = {
    "preprocess_id": 1,
    "inference_id": 1,
    "model_path" : "./yolov7-forklift-person.dxnn"
}
with open("yolov7-forklift-person.json", "w") as f: json.dump(yolov7_custom, f, indent=2)

### 6. Copy a video file

In [ ]:
!cp ../../../assets/forklift-worker.mp4 ./

### 7. Run the following DX-STREAM pipeline

In [ ]:
# video file as an input
VIDEO_SRC='forklift-worker.mp4'

# GStreamer pipeline configuration
!gst-launch-1.0 filesrc location=$VIDEO_SRC ! decodebin ! \
                   dxpreprocess config-file-path=dx_stream/configs/Object_Detection/YoloV7/preprocess_config.json ! queue ! \
                   dxinfer config-file-path=yolov7-forklift-person.json ! queue ! \
                   dxpostprocess config-file-path=dx_stream/configs/Object_Detection/YoloV7/postprocess_config.json ! queue ! \
                   dxosd width=1280 height=720 ! queue ! \
                   videoconvert ! fpsdisplaysink sync=false

## Debug Tips

Dump GSTreamer pipiline configurations as an image:

In [ ]:
!sudo apt install -y graphviz

In [ ]:
!export GST_DEBUG_DUMP_DOT_DIR=./ && ./run_demo.sh <<< 0

In [ ]:
#!dot -Tpng -o NULL_READY.png *NULL_READY.dot
#!dot -Tpng -o READY_PAUSED.png *READY_PAUSED.dot
!dot -Tpng -o PAUSED_PLAYING.png *PAUSED_PLAYING.dot
#!dot -Tpng -o PLAYING_PAUSED.png *PLAYING_PAUSED.dot
# -Tpng : output format is 'png'
# -o : output file name
# *.dot : input file name

In [ ]:
from IPython.display import Image, display

# Show the image
display(Image(filename="PAUSED_PLAYING.png"))